### Scope: Have a list of cities and return the average maximum temperature predicted for the next 7 days (including the current day).






Description:
The initial request was to use the MetaWeather API, but it is no longer working, and all callbacks returned errors.
According to my research the API was turned off.

The solution was pivoted to another public-use API, open-meteo (https://open-meteo.com/en/docs?), which is functional and contains the necessary information for the initial project request.

Below is the code used to fulfill the scope for three cities: Salt Lake City, Los Angeles, and Boise, Idaho.

The latitude and longitude of each city are required; this can be found on the API provider's website, as per the link above.

By providing a list of city names, latitude, and longitude, the "get_weekly_avg_high" function is used. This function parallelizes API calls, allowing up to 5 workers to be called simultaneously. The "fetch_city_weather" function queries the API, extracts the data, and calculates the average maximum temperature for the next 7 days for each city.

The result is a dataframe with the features city name, latitude, longitude, and average_maximum_temperature in °C.

Above each snippet is a brief description of its function.

**Import of libraries used as API query, data manipulation and processing parallelism.**

In [1]:
import requests
import pandas as pd
from collections import defaultdict
from datetime import datetime
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed

**Definition of the API call function with security warnings in case of any deviation from the expected result.**

In [2]:
def fetch_city_weather(city: str, lat: float, lon: float):
    try:
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "hourly" not in data or "time" not in data["hourly"] or "temperature_2m" not in data["hourly"]:
            return (city, lat, lon, "⚠️ Missing fields")

        times = data["hourly"]["time"]
        temps = data["hourly"]["temperature_2m"]

        if not times or not temps or len(times) != len(temps):
            return (city, lat, lon, "⚠️ Incomplete data")

        daily_temps = defaultdict(list)
        for t, temp in zip(times, temps):
            date = datetime.fromisoformat(t).date()
            daily_temps[date].append(temp)

        if not daily_temps:
            return (city, lat, lon, "⚠️ No daily data")

        daily_highs = {date: max(values) for date, values in daily_temps.items()}
        first_week = list(daily_highs.values())[:7]
        if not first_week:
            return (city, lat, lon, "⚠️ No 7-day forecast")

        avg_high = sum(first_week) / len(first_week)
        return (city, lat, lon, round(avg_high, 1))

    except requests.exceptions.RequestException as e:
        return (city, lat, lon, f"⚠️ Request failed: {e}")
    except Exception as e:
        return (city, lat, lon, f"⚠️ Unexpected error: {e}")

**Definition of the function that invokes the caller function in parallel and groups the results into a dataframe.**

In [3]:
def get_weekly_avg_high(locations: List[Tuple[str, float, float]]) -> pd.DataFrame:
    results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_city = {executor.submit(fetch_city_weather, city, lat, lon): (city, lat, lon) for city, lat, lon in locations}

        for future in as_completed(future_to_city):
            result = future.result()
            results.append(result)

    df = pd.DataFrame(results, columns=["City", "Latitude", "Longitude", "Avg_Weekly_High °C"])
    return df

**Creating the list of cities that will be used in the API query.**

In [4]:
cities = [
        ("Salt Lake City", 40.7608, -111.8911),
        ("Los Angeles", 34.0522, -118.2437),
        ("Boise",43.6135, -116.2035)
    ]  

**Function invocation.**

In [5]:
weekly_avgs = get_weekly_avg_high(cities)

**End result.**

In [6]:
weekly_avgs

,City,Latitude,Longitude,Avg_Weekly_High °C
0,Salt Lake City,40.7608,-111.8911,34.6
1,Boise,43.6135,-116.2035,36.9
2,Los Angeles,34.0522,-118.2437,34.5
